In [86]:
import pandas as pd
import numpy as np
# from matplotlib import pyplot as plt
# import boto3, re, sys, math, json, os, sagemaker, urllib.request, IPython
import joblib
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
# from sklearn import *
import pickle
from azure.storage.blob import BlobServiceClient

In [ ]:
# STORAGEACCOUNTURL= "https://tst18220095.blob.core.windows.net/"
# STORAGEACCOUNTKEY= "Vp0M51+EQMqigMTzWWDXUQpgPy5FEvoN0i82swDHV9HZNqdv6GEgUPClTwtr6hFR3/poVTtmt7kV+AStkc6dmg=="
# LOCALFILENAME= "Cleaned_2018_Flights1.csv"
# CONTAINERNAME= "newcontainer"
# BLOBNAME= "Cleaned_2018_Flights.csv"

# #download from blob
# blob_service_client_instance = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)
# blob_client_instance = blob_service_client_instance.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)
# with open(LOCALFILENAME, "wb") as my_blob:
#     blob_data = blob_client_instance.download_blob()
#     blob_data.readinto(my_blob)

In [97]:
pip freeze

aiosqlite==0.17.0
anyio==3.6.2
appdirs @ file:///home/conda/feedstock_root/build_artifacts/appdirs_1603108395799/work
appnope @ file:///Users/runner/miniforge3/conda-bld/appnope_1635819655903/work
argon2-cffi @ file:///Users/runner/miniforge3/conda-bld/argon2-cffi_1636021455522/work
async-generator==1.10
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1620387926260/work
azure-core==1.26.1
azure-storage-blob==12.14.1
Babel @ file:///home/conda/feedstock_root/build_artifacts/babel_1619719576210/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backports.functools_lru_cache_1618230623929/work
bcrypt==4.0.1
bleach @ file:///home/conda/feedstock_root/build_artifacts/bleach_1629908509068/work
boto3==1.26.25
botocore==1.29.25
Bottleneck @ file:///opt/concourse/worker/volumes/live/ac8c8ef3-2ed0-42e9-6ec0-5bb05ad938f6/volume/bottleneck_1607575111469/work


Note: you may need to restart the kernel to use updated packages.


In [91]:
data = pd.read_csv('Cleaned_2018_Flights1.csv')
data = data.dropna()
data = data.drop(
    columns=[
        "Unnamed: 0",
        "ItinID",
        "MktID",
        "MktCoupons",
        "OriginWac",
        "DestWac",
        "ContiguousUSA",
        "Miles"
    ]
)
data.head()

,Quarter,Origin,Dest,NumTicketsOrdered,AirlineCompany,PricePerTicket
0,1,PHL,LAX,1.0,AA,672.87
1,1,PHL,LAX,1.0,AA,367.68
2,1,PHL,LAX,1.0,AA,417.94
3,1,PHL,LAX,1.0,AA,247.10
4,1,PHL,LAX,1.0,AA,276.35


In [41]:
y = data[["PricePerTicket"]]
x = data.drop(columns=["PricePerTicket"])

In [44]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42
)

In [50]:
numerical_cols = ["NumTicketsOrdered"]
categorical_cols = ["Quarter", "Origin", "Dest", "AirlineCompany"]

categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
)
preprocessor = ColumnTransformer(
    transformers=[("cat", categorical_transformer, categorical_cols)],
    remainder="passthrough",
)

In [92]:
LR_pipe = Pipeline([("preprocessor", preprocessor), ("LinearRegressor", LinearRegression())])
LR_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Quarter', 'Origin', 'Dest',
                                                   'AirlineCompany'])])),
                ('LinearRegressor', LinearRegression())])

In [93]:
joblib.dump(LR_pipe, "API/models/flightmodel.joblib")

['API/models/flightmodel.joblib']

file = "lr_model.sav"
pickle.dump(lr, open(file, 'wb'))

file = "preprocerssor.sav"
pickle.dump(preprocessor, open(file, 'wb'))

TEST 1

In [122]:
# key1 = open("preprocerssor.sav","rb")
# key2 = open("lr_model.sav","rb")
key3 = open("API/models/flightmodel.joblib","rb")
model = joblib.load(key3)
test_datas = [1,"BOS","LAX",2,"AA"]

In [118]:
plan = "4,SFO,LAX,4,DL"
types = {
    "Quarter": np.float64,
    "Origin": str,
    "Dest": str,
    "NumTicketsOrdered": np.float64,
    "AirlineCompany": str,
}
    
test_data = plan.split(",")
df = pd.DataFrame(test_data).transpose()
df.columns = list(types.keys())
df = df.astype(types)
df.to_numpy()

array([[4.0, 'SFO', 'LAX', 4.0, 'DL']], dtype=object)

In [120]:
test_data

['4', 'SFO', 'LAX', '4', 'DL']

In [124]:
test_datas
df = pd.DataFrame(test_datas).transpose()
df.columns = list(types.keys())
df = df.astype(types)
df.to_numpy()

array([[1.0, 'BOS', 'LAX', 2.0, 'AA']], dtype=object)

In [126]:
model.predict(df)[0][0]

289.0336788164434

In [59]:
data["AirlineCompany"].unique()

array(['AA', 'AS', 'DL', 'UA', 'B6', 'SY', 'WN', 'F9', 'G4', 'HA', 'NK',
       'VX'], dtype=object)

In [62]:
data["Origin"].unique()

array(['PHL', 'LAX', 'SAN', 'SFO', 'AUS', 'BOS', 'MIA', 'ORD', 'LGA',
       'CLT', 'TPA', 'DFW', 'LAS', 'MCI', 'MCO', 'FLL', 'MSY', 'PBI',
       'PHX', 'RSW', 'MEM', 'MHT', 'SJU', 'SLC', 'MSP', 'SAT', 'CLE',
       'DEN', 'JAC', 'SEA', 'RDU', 'SMF', 'DTW', 'PIT', 'PVD', 'ATL',
       'BNA', 'DCA', 'STL', 'ABQ', 'BOI', 'JFK', 'KOA', 'BWI', 'EWR',
       'JAX', 'CMH', 'OMA', 'PDX', 'COS', 'DSM', 'GEG', 'HDN', 'IND',
       'MKE', 'MTJ', 'FAT', 'HNL', 'LIH', 'IAH', 'SJC', 'SNA', 'OGG',
       'RNO', 'OAK', 'ONT', 'PSP', 'RIC', 'SAV', 'PNS', 'SDF', 'PWM',
       'BDL', 'TUL', 'ORF', 'VPS', 'TUS', 'SBA', 'ILM', 'IAD', 'CVG',
       'GSP', 'STT', 'STX', 'SYR', 'BUF', 'EGE', 'ICT', 'LBB', 'ADK',
       'ANC', 'ADQ', 'JNU', 'BET', 'BRW', 'CDV', 'KTN', 'FAI', 'PSG',
       'WRG', 'SIT', 'OME', 'OTZ', 'SCC', 'SRQ', 'BIL', 'BZN', 'AGS',
       'ALB', 'CHS', 'GSO', 'OKC', 'LEX', 'HOU', 'ROA', 'TLH', 'LFT',
       'CRW', 'FNT', 'LIT', 'JAN', 'BTR', 'GPT', 'BHM', 'DAL', 'MDW',
       'MDT', 'EVV',